# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [167]:
#Importing relevant libraries and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.compose import ColumnTransformer
from category_encoders import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

#Setting display settings
pd.set_option('display.max_columns', None)

#Importing file with pd.read_csv
house_sales = pd.read_csv("house_sales.csv")

#Checking missing values in city
print(house_sales["city"].value_counts())

#Calculating missing/miscoded values in city
missing_city = (house_sales["city"] == "--").sum()

#Printing missing city to ensure it is correct
print(missing_city)

Silvertown    517
Teasdale      366
Poppleton     362
Riverford     182
--             73
Name: city, dtype: int64
73


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [168]:
#Creating wrangler function
def wrangle(data):

    #Correcting miscoded values
    house_sales["city"] = house_sales["city"].apply(lambda x: "Unknown" if x=="--" else x)
    house_sales["house_type"] = house_sales["house_type"].apply(lambda x: "Terraced" if x=="Terr." else x)
    house_sales["house_type"] = house_sales["house_type"].apply(lambda x: "Semi-detached" if x=="Semi" else x)
    house_sales["house_type"] = house_sales["house_type"].apply(lambda x: "Detached" if x=="Det." else x)
    house_sales["area"] = house_sales["area"].apply(lambda x: x.split()[0])
    
    #Correcting incorrect data types
    house_sales["house_id"] = house_sales["house_id"].astype(str)
    house_sales["area"] = house_sales["area"].astype(float)
    
    #Imputing missing values
    house_sales.dropna(subset=["sale_price"], inplace=True)
    house_sales["sale_date"].fillna("2023-01-01", inplace=True)
    house_sales["months_listed"].fillna(house_sales["months_listed"].mean().round(1), inplace=True)
    house_sales["bedrooms"].fillna(house_sales["bedrooms"].mean().round(0), inplace=True) 
    house_sales["area"].fillna(house_sales["area"].mean().round(1), inplace=True)
    house_sales["house_type"].fillna(house_sales["house_type"].mode()[0], inplace=True)
    
    #Encoding nominal categorical columns 
    nominal_encoder = OneHotEncoder()
    nominal_encoder.fit_transform(house_sales[["house_id", "city"]])
    
    #Encoding ordinal categorical columns
    label_encoder = OrdinalEncoder()
    label_encoder.fit_transform(house_sales[["house_type", "sale_date"]])

    return house_sales

#Running wrangler
clean_data = wrangle(house_sales)

#Checking every condition is met in clean_data
print(clean_data.head(10))
print(clean_data.dtypes)
print(clean_data.isna().any())
print((house_sales["sale_price"] < 0).sum())

  house_id        city  sale_price   sale_date  months_listed  bedrooms  \
0  1217792  Silvertown       55943  2021-09-12            5.4         2   
1  1900913  Silvertown      384677  2021-01-17            6.3         5   
2  1174927   Riverford      281707  2021-11-10            6.9         6   
3  1773666  Silvertown      373251  2020-04-13            6.1         6   
4  1258487  Silvertown      328885  2020-09-24            8.7         5   
5  1539789   Riverford       47143  2020-09-25            5.1         2   
6  1353069    Teasdale      429914  2021-05-18            5.9         6   
7  1155864    Teasdale      284440  2020-10-15            5.8         4   
8  1608726    Teasdale       87134  2021-01-07            4.4         2   
9  1358764   Poppleton      211203  2020-05-04            3.0         4   

      house_type   area  
0  Semi-detached  107.8  
1       Detached  498.8  
2       Detached  542.5  
3       Detached  528.4  
4       Detached  477.1  
5  Semi-detached  

# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [169]:
#Building reviews_by_reting df with Reviews, and Rating columns
price_by_rooms = house_sales[["bedrooms", "sale_price"]]

#Using groupby and agreggation to obtain median, min, and max values of Reviews for each unique rating
price_by_rooms = price_by_rooms.groupby("bedrooms").agg({"sale_price": ["mean", "var"]}).round(1).reset_index()

#Setting column names according to requirements
price_by_rooms.columns = ["bedrooms", "avg_price", "var_price"]

#Viewing df to ensure everything is ok
print(price_by_rooms)

   bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [170]:
#Importing training and validation data
train = pd.read_csv("train.csv")
X_test = pd.read_csv("validation.csv")

#Checking training and validation datasets
print(train.head(10))
print(X_test.head(10))

#Checking datatypes of columns in both dataframes
print(train.dtypes)
print(X_test.dtypes)

#Storing place names for later use
house_ids = X_test["house_id"]

#Separating features and target for training and testing
X_train = train.drop(["house_id", "sale_price"], axis=1)
y_train = train["sale_price"]
X_test.drop("house_id", axis =1, inplace=True)

#Columns to encode, split between ordinal encoding, non-ordinal encoding, and columns to passthrough
ohe_cols = ["city"]
oe_cols = ["sale_date", "house_type"]
non_enc_columns = ["months_listed", "bedrooms", "area"]

#Encoding nominal variables
encoder = ColumnTransformer(
    transformers=[('ohe', OneHotEncoder(), ohe_cols), ("oe", OrdinalEncoder(), oe_cols), ('num', 'passthrough', non_enc_columns)])

#Building function to run k-fold cross validation to get MSE of baseline model, using training subset
def bs_mse(feats, target):
    
    #Encoding categorical variables
    enc_feats = encoder.fit_transform(feats)
    
    #Initializaing LogReg
    tr_model = LinearRegression()
    
    #Defining cross-validation parameters
    cross_val = KFold(n_splits=5, shuffle=True)
    
    #Obtaining scores for different splits
    scores = cross_val_score(tr_model, enc_feats, target, cv=cross_val, scoring='neg_mean_squared_error')
    
    #Extracting the mean MSE from the folds
    return math.sqrt(-scores.mean())

#Building data pipeline for baseline model
pipeline = Pipeline([('encoder', encoder),('baseline', LinearRegression())
])

#Fitting the model to the training data
pipeline.fit(X_train, Y_train)

#Predicting reviews using baseline model
predictions = pipeline.predict(X_test)

base_result = pd.DataFrame({"house_id": house_ids, "price": predictions})

print(base_result.head(20))
print(bs_mse(X_train, y_train))

   house_id        city  sale_price   sale_date  months_listed  bedrooms  \
0   1634561    Teasdale      401869  2021-12-14            7.0         6   
1   1009770  Silvertown      372387  2022-09-11            8.1         6   
2   1946667  Silvertown      325473  2020-08-19            5.4         5   
3   1798290  Silvertown      349469  2022-12-10            6.4         5   
4   1533461   Poppleton      199995  2020-04-07            4.3         4   
5   1232200    Teasdale       67372  2021-05-09            5.9         2   
6   1182761   Poppleton      193803  2022-10-30            9.5         4   
7   1346363   Poppleton      321844  2020-12-06            6.3         6   
8   1459745   Riverford      268723  2022-09-07            5.8         6   
9   1610313   Poppleton      113063  2021-07-19            2.2         3   

      house_type   area  
0       Detached  519.7  
1       Detached  507.8  
2       Detached  466.8  
3       Detached  499.4  
4       Detached  335.0  
5  Semi

# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [148]:
#Importing training and validation data
train = pd.read_csv("train.csv")
X_test = pd.read_csv("validation.csv")

#Checking training and validation datasets
print(train.head(10))
print(X_test.head(10))

#Checking datatypes of columns in both dataframes
print(train.dtypes)
print(X_test.dtypes)

#Storing place names for later use
house_ids = X_test["house_id"]

#Separating features and target for training and testing
X_train = train.drop(["house_id", "sale_price"], axis=1)
y_train = train["sale_price"]
X_test.drop("house_id", axis =1, inplace=True)

#Columns to encode, split between ordinal encoding, non-ordinal encoding, and columns to passthrough
ohe_cols = ["city"]
oe_cols = ["sale_date", "house_type"]
non_enc_columns = ["months_listed", "bedrooms", "area"]

#Encoding nominal variables
encoder = ColumnTransformer(
    transformers=[('ohe', OneHotEncoder(), ohe_cols), ("oe", OrdinalEncoder(), oe_cols), ('num', 'passthrough', non_enc_columns)])

#Building function to run k-fold cross validation to get MSE of Comparison model, using training subset
def cp_mse(feats, target):
    
    #Creating mse_list to store mse's for different parameters, pred_list to store parameters
    mse_list=[]
    param_list=[]
    
    #Encoding categorical variables
    enc_feats = encoder.fit_transform(feats)
    
    #Splitting train and test data within train dataset to assess performance
    ft_train, ft_test, tg_train, tg_test = train_test_split(feats, target, test_size=0.75)
    
    #Iterating hyperparameters to find optimal combination
    for max_depth in range(3, 6):
       for n_estimators in range(50, 200):
            for learning_rate in np.arange(0.01, 0.05, 0.01):
                
                #Initializing LogReg
                tr_model = GradientBoostingRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)
                
                #Fitting the model to the training data
                pipeline.fit(ft_train, tg_train)

                #Predicting reviews using baseline model
                y_pred = pipeline.predict(ft_test)
                
                #Appending parameters on param_list
                param_list.append((n_estimators, max_depth, learning_rate))
                
                #Calculating MSE for each set of parameters
                mse = mean_squared_error(tg_test, y_pred)
                
                #Appending mse values in mse_list
                mse_list.append(math.sqrt(mse))
                
                #Finding parameters that provide the lowest mse
                lowest_mse = mse_list.index(min(mse_list))
                best_parameters = param_list[lowest_mse]
                
    return best_parameters, min(mse_list)

#Storing function results to recurring function calls
best_parameters = cp_mse(X_train, y_train)[0]
min_mse = cp_mse(X_train, y_train)[1]

#Defining the pipeline steps
pipeline = Pipeline([('encoder', encoder),('comparison', GradientBoostingRegressor(n_estimators=best_parameters[0], max_depth=best_parameters[1], learning_rate=best_parameters[2]))])

#Fitting the model to the training data
pipeline.fit(X_train, y_train)

#Predicting reviews using baseline model
predictions = pipeline.predict(X_test)

compare_result = pd.DataFrame({"house_id": house_ids, "price": predictions})
print(best_parameters)
print(min_mse)
print(compare_result.head(20))

   house_id        city  sale_price   sale_date  months_listed  bedrooms  \
0   1634561    Teasdale      401869  2021-12-14            7.0         6   
1   1009770  Silvertown      372387  2022-09-11            8.1         6   
2   1946667  Silvertown      325473  2020-08-19            5.4         5   
3   1798290  Silvertown      349469  2022-12-10            6.4         5   
4   1533461   Poppleton      199995  2020-04-07            4.3         4   
5   1232200    Teasdale       67372  2021-05-09            5.9         2   
6   1182761   Poppleton      193803  2022-10-30            9.5         4   
7   1346363   Poppleton      321844  2020-12-06            6.3         6   
8   1459745   Riverford      268723  2022-09-07            5.8         6   
9   1610313   Poppleton      113063  2021-07-19            2.2         3   

      house_type   area  
0       Detached  519.7  
1       Detached  507.8  
2       Detached  466.8  
3       Detached  499.4  
4       Detached  335.0  
5  Semi